In [1]:
# Imports
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import numpy as np
from matplotlib import pyplot as plt
from sklearn import metrics
from sklearn.utils import resample
import scipy.stats as st

import sys
sys.path.append('../')
from lqtnet import stats
from lqtnet import train

In [2]:
# Load data, model, and make predictions

MODEL_PATH = '../models/2023.01.30-11/'
METADATA_PATH = '../metadata/ecg_metadata_2023jan16_final.csv'
ECG_SOURCE_DIR = '../ecgs/csv_normalized_2500/'

(_, x_intval, x_extval, _, y_intval_true, y_extval_true) = train._import_data(
    metadata_path=METADATA_PATH,
    ecg_source_dir=ECG_SOURCE_DIR)

model = train._load_model(MODEL_PATH)

y_intval_pred = model.predict(x_intval)
y_extval_pred = model.predict(x_extval)



Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-02-25 14:54:04.975163: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-25 14:54:04.975594: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-02-25 14:54:06.895011: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-25 14:54:06.996555: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


6/6 [==============================] - 1s 95ms/step


In [10]:
y_true = np.where(y_extval_true[:,0]==0,1,0)
y_probas = 1-y_extval_pred[:,0]

def auc_ci(y_true, y_probas, n_samples):
    auc = []
    for _ in range(n_samples):
        y_true_sample, y_probas_sample = resample(y_true, y_probas, replace=True)
        auc.append(metrics.roc_auc_score(y_true_sample, y_probas_sample))

    mean = np.mean(auc)
    ci = st.norm.interval(confidence=0.95, loc=mean, scale=st.sem(auc))
    result = f"{mean:.3f} ({ci[0]:.3f}-{ci[1]:.3f})"
    return(result)

print(auc_ci(y_true, y_probas, 10))
print(auc_ci(y_true, y_probas, 100))
print(auc_ci(y_true, y_probas, 1000))
print(auc_ci(y_true, y_probas, 10000))


0.931 (0.923-0.939)
0.921 (0.917-0.925)
0.922 (0.921-0.924)
0.923 (0.922-0.923)
0.922 (0.922-0.922)


In [5]:
st.norm.interval(confidence=0.95,
                    loc=np.mean(auc),
                    scale=st.sem(auc))

(0.9217892342572381, 0.924251931133928)

In [ ]:
# Internal Validation
# True = LQTS1/2
# False = Control
y_true = np.where(y_intval_true[:,0]==0,1,0)
y_probas = 1-y_intval_pred[:,0]

t = stats.best_sen_thresh(y_true, y_probas)
stats.roc_pr_curves(
    y_true, y_probas,
    thresh=t,
    title=f"LQTS1/2 carrier status (Internal Validation), Threshold={t:.2f}",
    labels=['Control', 'LQTS1/2'],
)

t = stats.youden_thresh(y_true, y_probas)
stats.roc_pr_curves(
    y_true, y_probas,
    thresh=t,
    title=f"LQTS1/2 carrier status (Internal Validation), Threshold={t:.2f}",
    labels=['Control', 'LQTS1/2'],
)

# # External Validation
y_true = np.where(y_extval_true[:,0]==0,1,0)
y_probas = 1-y_extval_pred[:,0]

t = stats.best_sen_thresh(y_true, y_probas)
stats.roc_pr_curves(
    y_true, y_probas,
    thresh=t,
    title=f"LQTS1/2 carrier status (External Validation), Threshold={t:.2f}",
    labels=['Control', 'LQTS1/2'],
)

t = stats.youden_thresh(y_true, y_probas)
stats.roc_pr_curves(
    y_true, y_probas,
    thresh=t,
    title=f"LQTS1/2 carrier status (External Validation), Threshold={t:.2f}",
    labels=['Control', 'LQTS1/2'],
)


In [ ]:
# LQTS1 vs LQTS2, Internal Validation
y_true, y_probas = stats.lqt1_probas(y_intval_true, y_intval_pred)
t = stats.youden_thresh(y_true, y_probas)
stats.roc_pr_curves(
    y_true, y_probas,
    thresh=t,
    title=f"LQTS type 1 vs type 2 (Internal Validation), Threshold={t:.2f}",
    labels=['LQTS type 1', 'LQTS type 2'],
)

# LQTS1 vs LQTS2, External Validation
y_true, y_probas = stats.lqt1_probas(y_extval_true, y_extval_pred)
t = stats.youden_thresh(y_true, y_probas)
stats.roc_pr_curves(
    y_true, y_probas,
    thresh=t,
    title=f"LQTS type 1 vs type 2 (External Validation), Threshold={t:.2f}",
    labels=['LQTS type 1', 'LQTS type 2'],
)